# Chapter 20: Intelligent Troubleshooting Agents

This notebook demonstrates how to build AI agents for network troubleshooting.

**What you'll learn:**
- Build agents that autonomously run diagnostic commands
- Create multi-stage troubleshooting systems
- Implement conversational agents with memory
- Add production safety features

**Note:** This notebook uses simulated network outputs for demonstration.
In production, you'd connect to real devices using Netmiko/NAPALM.

## Setup: Install Dependencies

In [ ]:
# Install required packages
!pip install -q langchain langchain-anthropic pydantic

In [ ]:
# Import libraries
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_anthropic import ChatAnthropic
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser
from langchain.memory import ConversationBufferMemory
from typing import Type, List, Dict, Tuple
from enum import Enum
from pydantic import BaseModel as PydanticBaseModel
import re
import json

print("✓ All libraries imported successfully")

In [ ]:
# Set your Anthropic API key
import os
from getpass import getpass

if 'ANTHROPIC_API_KEY' not in os.environ:
    os.environ['ANTHROPIC_API_KEY'] = getpass('Enter your Anthropic API key: ')

print("✓ API key configured")

---

## Part 1: Build Diagnostic Tools

Tools are the "hands" of the agent - they allow it to take actions.

We'll create three essential troubleshooting tools:
1. **show_command** - Run Cisco show commands
2. **ping** - Test connectivity
3. **traceroute** - Trace network path

In [ ]:
# Simulated network device outputs (in production: use Netmiko)
SIMULATED_OUTPUTS = {
    "show ip interface brief": """
Interface              IP-Address      OK? Method Status                Protocol
GigabitEthernet0/0     10.1.1.1        YES manual up                    up
GigabitEthernet0/1     10.2.2.1        YES manual up                    down
GigabitEthernet0/2     192.168.1.1     YES manual up                    up
Loopback0              1.1.1.1         YES manual up                    up
""",
    "show ip route": """
Gateway of last resort is 192.168.1.254 to network 0.0.0.0

      10.0.0.0/8 is variably subnetted, 2 subnets
C        10.1.1.0/24 is directly connected, GigabitEthernet0/0
C        10.2.2.0/24 is directly connected, GigabitEthernet0/1
S*    0.0.0.0/0 [1/0] via 192.168.1.254
""",
    "show ip bgp summary": """
BGP router identifier 1.1.1.1, local AS number 65001

Neighbor        V    AS MsgRcvd MsgSent   TblVer  InQ OutQ Up/Down  State/PfxRcd
192.168.1.2     4 65002    1234    1235       45    0    0 00:15:23        150
203.0.113.1     4 65003       0       0        0    0    0 never    Idle
""",
    "show interface gigabitethernet0/1": """
GigabitEthernet0/1 is up, line protocol is down
  Hardware is iGbE, address is 0000.0c07.ac01
  Internet address is 10.2.2.1/24
  MTU 1500 bytes, BW 1000000 Kbit/sec
  Full-duplex, 1000Mb/s, media type is RJ45
  Last input never, output 00:00:01, output hang never
  Input queue: 0/75/0/0 (size/max/drops/flushes); Total output drops: 0
  5 minute input rate 0 bits/sec, 0 packets/sec
  5 minute output rate 0 bits/sec, 0 packets/sec
     0 packets input, 0 bytes, 0 no buffer
     0 input errors, 0 CRC, 0 frame, 0 overrun, 0 ignored
     13 packets output, 1234 bytes, 0 underruns
     0 output errors, 0 collisions, 1 interface resets
"""
}

print("✓ Simulated device outputs loaded")

In [ ]:
# Tool 1: Show Command Tool
class ShowCommandInput(BaseModel):
    """Input schema for show command."""
    command: str = Field(description="Show command to run (must start with 'show')")

class ShowCommandTool(BaseTool):
    """Execute show commands on network devices."""
    name: str = "show_command"
    description: str = """
    Execute read-only show commands on network devices.
    Examples: 'show ip interface brief', 'show ip route', 'show ip bgp summary'
    """
    args_schema: Type[BaseModel] = ShowCommandInput

    def _run(self, command: str) -> str:
        """Run show command (simulated for demo)."""
        # Safety check
        if not command.strip().lower().startswith("show"):
            return "ERROR: Only 'show' commands are allowed"
        
        # Get simulated output
        output = SIMULATED_OUTPUTS.get(
            command.lower(),
            f"[Simulated output for: {command}]\nCommand executed successfully."
        )
        return output

# Tool 2: Ping Tool
class PingInput(BaseModel):
    """Input schema for ping."""
    host: str = Field(description="IP address or hostname to ping")

class PingTool(BaseTool):
    """Ping a host to test connectivity."""
    name: str = "ping"
    description: str = "Test network connectivity by pinging an IP or hostname"
    args_schema: Type[BaseModel] = PingInput

    def _run(self, host: str) -> str:
        """Execute ping (simulated)."""
        return f"""
PING {host}: 4 packets transmitted, 4 received, 0% packet loss
round-trip min/avg/max = 1.2/2.3/3.4 ms
"""

# Tool 3: Traceroute Tool
class TracerouteInput(BaseModel):
    """Input schema for traceroute."""
    destination: str = Field(description="Destination IP or hostname")

class TracerouteTool(BaseTool):
    """Trace network path to destination."""
    name: str = "traceroute"
    description: str = "Trace the network path to identify where packets are dropped"
    args_schema: Type[BaseModel] = TracerouteInput

    def _run(self, destination: str) -> str:
        """Execute traceroute (simulated)."""
        return f"""
traceroute to {destination}:
 1  192.168.1.1  1.2 ms
 2  10.5.5.1     2.3 ms
 3  {destination} 3.4 ms
"""

# Combine all tools
def get_troubleshooting_tools():
    """Get list of all troubleshooting tools."""
    return [ShowCommandTool(), PingTool(), TracerouteTool()]

print("✓ Troubleshooting tools created")
print(f"  - {ShowCommandTool().name}")
print(f"  - {PingTool().name}")
print(f"  - {TracerouteTool().name}")

---

## Part 2: Basic Troubleshooting Agent

This agent can:
- Understand natural language problem descriptions
- Decide which commands to run
- Analyze outputs
- Identify root causes

In [ ]:
class BasicTroubleshootingAgent:
    """Simple troubleshooting agent."""
    
    def __init__(self, api_key: str):
        """Initialize the agent."""
        # LLM (the "brain")
        self.llm = ChatAnthropic(
            model="claude-3-5-sonnet-20241022",
            api_key=api_key,
            temperature=0.0  # Deterministic
        )
        
        # Tools (the "hands")
        self.tools = get_troubleshooting_tools()
        
        # Prompt (the "instructions")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert network troubleshooting assistant.

Your goal: Diagnose network issues systematically.

Process:
1. Understand the symptom
2. Form a hypothesis
3. Run diagnostic commands
4. Analyze results
5. Identify root cause
6. Suggest specific fix

Be systematic and explain your reasoning."""),
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad")
        ])
        
        # Create agent
        agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        
        # Agent executor
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            max_iterations=10
        )
    
    def troubleshoot(self, problem: str) -> str:
        """Troubleshoot a network problem."""
        result = self.agent_executor.invoke({"input": problem})
        return result['output']

print("✓ BasicTroubleshootingAgent class defined")

In [ ]:
# Test the basic agent
print("Creating basic troubleshooting agent...\n")
agent = BasicTroubleshootingAgent(api_key=os.environ['ANTHROPIC_API_KEY'])

print("="*70)
print("PROBLEM: Users on VLAN 20 (10.2.2.0/24) cannot access the network")
print("="*70)

result = agent.troubleshoot(
    "Users on VLAN 20 (10.2.2.0/24) cannot access the network"
)

print("\n" + "="*70)
print("AGENT'S ANALYSIS:")
print("="*70)
print(result)

---

## Part 3: Multi-Stage Troubleshooting Agent

This agent follows a structured methodology:
- Plans each stage based on OSI model
- Executes diagnostic commands
- Analyzes results
- Decides next stage based on findings

In [ ]:
# Define diagnostic stages
class DiagnosticStage(str, Enum):
    """Stages in network troubleshooting."""
    IDENTIFY_LAYER = "identify_layer"
    CHECK_PHYSICAL = "check_physical"
    CHECK_DATA_LINK = "check_data_link"
    CHECK_NETWORK = "check_network"
    CHECK_ROUTING = "check_routing"
    ROOT_CAUSE_FOUND = "root_cause_found"

# Define diagnostic plan structure
class DiagnosticPlan(PydanticBaseModel):
    """Plan for a diagnostic stage."""
    stage: str
    hypothesis: str
    commands_to_run: List[str]
    reasoning: str

print("✓ Diagnostic stages and plan structure defined")

In [ ]:
class MultiStageTroubleshooter:
    """Structured multi-stage troubleshooting agent."""
    
    def __init__(self, api_key: str):
        """Initialize multi-stage troubleshooter."""
        self.llm = ChatAnthropic(
            model="claude-3-5-sonnet-20241022",
            api_key=api_key,
            temperature=0.0
        )
        self.parser = JsonOutputParser(pydantic_object=DiagnosticPlan)
    
    def plan_next_stage(self, symptom: str, previous_findings: List[Dict] = None) -> DiagnosticPlan:
        """Plan the next diagnostic stage."""
        findings_text = "None yet" if not previous_findings else "\n".join([
            f"Stage {i+1}: {f['hypothesis'][:100]}..."
            for i, f in enumerate(previous_findings)
        ])
        
        prompt = ChatPromptTemplate.from_template("""
Plan the next troubleshooting stage.

Symptom: {symptom}
Previous findings: {findings}

Based on OSI model:
1. Identify which layer to investigate
2. Form a hypothesis
3. Choose diagnostic commands
4. Explain reasoning

Available stages: identify_layer, check_physical, check_data_link, 
check_network, check_routing, root_cause_found

{format_instructions}

JSON:""")
        
        response = self.llm.invoke(
            prompt.format(
                symptom=symptom,
                findings=findings_text,
                format_instructions=self.parser.get_format_instructions()
            )
        )
        
        plan_data = json.loads(response.content)
        return DiagnosticPlan(**plan_data)
    
    def execute_stage(self, plan: DiagnosticPlan) -> Dict:
        """Execute a diagnostic stage."""
        tool = ShowCommandTool()
        results = []
        
        for command in plan.commands_to_run:
            output = tool._run(command)
            results.append({"command": command, "output": output})
        
        return {
            "stage": plan.stage,
            "hypothesis": plan.hypothesis,
            "results": "\n\n".join([f"$ {r['command']}\n{r['output']}" for r in results])
        }
    
    def troubleshoot(self, symptom: str, max_stages: int = 3) -> Dict:
        """Perform complete multi-stage troubleshooting."""
        findings = []
        
        print(f"\n{'='*70}")
        print(f"MULTI-STAGE TROUBLESHOOTING")
        print(f"{'='*70}")
        print(f"Symptom: {symptom}\n")
        
        for stage_num in range(max_stages):
            print(f"\n[STAGE {stage_num + 1}]")
            
            # Plan
            plan = self.plan_next_stage(symptom, findings)
            print(f"Stage: {plan.stage}")
            print(f"Hypothesis: {plan.hypothesis}")
            print(f"Commands: {plan.commands_to_run}")
            
            if plan.stage == "root_cause_found":
                print("\n✓ Root cause identified!")
                break
            
            # Execute
            result = self.execute_stage(plan)
            findings.append(result)
            print(f"✓ Stage {stage_num + 1} complete")
        
        return {"symptom": symptom, "findings": findings}

print("✓ MultiStageTroubleshooter class defined")

In [ ]:
# Test multi-stage troubleshooting
multi_agent = MultiStageTroubleshooter(api_key=os.environ['ANTHROPIC_API_KEY'])

result = multi_agent.troubleshoot(
    "Users report intermittent connectivity issues",
    max_stages=3
)

---

## Part 4: Conversational Agent with Memory

This agent remembers the conversation and can handle follow-up questions.

In [ ]:
class ConversationalAgent:
    """Agent with conversation memory."""
    
    def __init__(self, api_key: str):
        """Initialize conversational agent."""
        self.llm = ChatAnthropic(
            model="claude-3-5-sonnet-20241022",
            api_key=api_key,
            temperature=0.2
        )
        
        # Memory - remembers conversation
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="output"
        )
        
        self.tools = get_troubleshooting_tools()
        
        # Prompt with memory placeholder
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a network troubleshooting assistant.

Remember previous messages. Reference earlier findings when relevant.
Use tools to gather diagnostic information when needed."""),
            MessagesPlaceholder("chat_history"),  # Conversation history
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad")
        ])
        
        agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        
        self.agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            memory=self.memory,
            verbose=True
        )
    
    def chat(self, message: str) -> str:
        """Chat with the agent."""
        result = self.agent_executor.invoke({"input": message})
        return result['output']

print("✓ ConversationalAgent class defined")

In [ ]:
# Test conversational agent
conv_agent = ConversationalAgent(api_key=os.environ['ANTHROPIC_API_KEY'])

print("="*70)
print("CONVERSATIONAL TROUBLESHOOTING")
print("="*70)

# First message
print("\n[YOU]: Users on VLAN 20 can't access the internet\n")
response1 = conv_agent.chat("Users on VLAN 20 can't access the internet")
print(f"[AGENT]: {response1}")

# Follow-up (agent remembers!)
print("\n[YOU]: Why is that interface down?\n")
response2 = conv_agent.chat("Why is that interface down?")
print(f"[AGENT]: {response2}")

# Another follow-up
print("\n[YOU]: How do I fix it?\n")
response3 = conv_agent.chat("How do I fix it?")
print(f"[AGENT]: {response3}")

---

## Part 5: Safety Features

Production agents need safety features to prevent dangerous operations.

In [ ]:
class SafeCommandValidator:
    """Validate commands before execution."""
    
    SAFE_COMMANDS = ["show", "ping", "traceroute", "display", "get"]
    
    DANGEROUS_PATTERNS = [
        r'\bno\b',
        r'\bshutdown\b',
        r'\breload\b',
        r'\bwrite\s+erase\b',
        r'\bdelete\b',
        r'\bconfigure\b',
    ]
    
    @classmethod
    def is_safe(cls, command: str) -> Tuple[bool, str]:
        """Check if command is safe."""
        cmd_lower = command.lower().strip()
        
        # Check safe prefix
        if not any(cmd_lower.startswith(safe) for safe in cls.SAFE_COMMANDS):
            return False, f"Command must start with: {', '.join(cls.SAFE_COMMANDS)}"
        
        # Check dangerous patterns
        for pattern in cls.DANGEROUS_PATTERNS:
            if re.search(pattern, cmd_lower):
                return False, f"Contains dangerous pattern: {pattern}"
        
        return True, "Safe"

print("✓ SafeCommandValidator class defined")

In [ ]:
# Test safety validation
test_commands = [
    "show ip interface brief",  # Safe
    "configure terminal",        # Dangerous
    "reload in 5",               # Dangerous
    "show running-config",       # Safe
    "no shutdown",               # Dangerous
]

print("\nCommand Safety Tests:")
print("="*70)

for cmd in test_commands:
    is_safe, reason = SafeCommandValidator.is_safe(cmd)
    status = "✓ SAFE" if is_safe else "✗ BLOCKED"
    print(f"{status}: {cmd}")
    if not is_safe:
        print(f"  Reason: {reason}")

---

## Summary

**What we built:**

1. **Basic Agent** - Autonomous troubleshooting with tools
2. **Multi-Stage Agent** - Structured investigation following OSI model
3. **Conversational Agent** - Memory for follow-up questions
4. **Safety Features** - Command validation to prevent dangerous operations

**Key Concepts:**

- **Tools** = Actions the agent can take
- **Memory** = Context across messages
- **Planning** = Structured methodology
- **Safety** = Validation and guardrails

**Production Considerations:**

- Replace simulated outputs with real Netmiko connections
- Add comprehensive audit logging
- Implement human-in-the-loop approval for changes
- Set rate limits and timeout controls
- Test thoroughly in lab environment first

**Next Steps:**

- Connect to real network devices
- Add more diagnostic tools (DNS, DHCP, etc.)
- Implement automated remediation
- Build incident response workflows